In [ ]:
"""
Purpose: To Run the neuron preprocessing


"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

# configuring the virtual module

In [2]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 30)
print(f"Sleeping {random_sleep_sec} sec before conneting")
time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

Sleeping 10 sec before conneting


INFO - 2020-11-22 07:57:53,819 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-22 07:57:53,820 - settings - Setting database.user to celiib
INFO - 2020-11-22 07:57:53,821 - settings - Setting database.password to newceliipass
INFO - 2020-11-22 07:57:53,825 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-22 07:57:53,826 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 07:57:53,838 - connection - Connect

Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons
Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-22 07:57:54,074 - settings - Setting enable_python_native_blobs to True


# Defining Our Table

In [3]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [4]:
from soma_extraction_utils import *
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 3454


In [5]:
#so that it will have the adapter defined
from datajoint_utils import *

In [6]:
#minnie.Decomposition.drop()
#schema.external['decomposition'].delete(delete_external_files=True)

In [7]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ---
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branche

    
    """

    key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0").proj())
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n----- Working on {segment_id}-------")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

        #3) Get the somas info *************************** Need to change this when actually run *******************
        somas = du.get_soma_mesh_list(segment_id) 
        print(f"somas = {somas}")
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"]

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")



        #7) Pass stats and file location to insert
        new_key = dict(key,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running The Populate

In [8]:
#(minnie.schema.jobs & "table_name='__decomposition'").delete()
#((schema.jobs & "table_name = '__decomposition'") & "timestamp>'2020-11-16 00:26:00'").delete()

In [ ]:
import time
import random
import compartment_utils as cu
cu = reload(cu)

start_time = time.time()
#time.sleep(random.randint(0, 900))
print('Populate Started')
Decomposition.populate(reserve_jobs=True, suppress_errors=False, order='random')
print('Populate Done')

print(f"Total time for Decomposition populate = {time.time() - start_time}")

Populate Started


INFO - 2020-11-22 07:58:44,571 - autopopulate - Found 39815 keys to populate
INFO - 2020-11-22 07:58:44,599 - connection - Transaction started
INFO - 2020-11-22 07:58:44,601 - autopopulate - Populating: {'segment_id': 864691135654027202, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}
INFO - 2020-11-22 07:58:44,605 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 07:58:44,793 - settings - Setting enable_python_native_blobs to True




----- Working on 864691135654027202-------


INFO - 2020-11-22 07:58:46,204 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 07:58:46,387 - settings - Setting enable_python_native_blobs to True


somas = [[<trimesh.Trimesh(vertices.shape=(8617, 3), faces.shape=(16548, 3))>], array([71.8887]), array([0.668])]

--- Beginning preprocessing of 864691135654027202---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(8617, 3), faces.shape=(16548, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(8617, 3), faces.shape=(16548, 3))>]
soma_mesh_list_centers = [array([ 522270.5532668 ,  640106.6644888 , 1067355.06928165])]
xvfb-run -n 3454 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Neuron_Processing/Practice/temp/neuron_99139.off -o /notebooks/Platinum_Datajoint/Neuron_Processing/Practice/temp/neuron_99139_fill_holes.off -s /notebooks/Platinum_Datajoint/Neuron_Processing/Practice/temp/fill_holes_393982.mls
removed temporary input file: /notebooks/Platinum_Datajoint/Neur

...




--- Working on 1-to-1 correspondence-----
max(original_labels),len(original_labels) = (2, 3)
empty_indices % = 0.09286892003297609
 conflict_indices % = 0.01916735366859027



AFTER face_lookup_resolved_test



Deleting branches from dictionary
MP_branches_with_stitch_point= [0]
MP_branches_for_correspondence = [0]
MP_leftover = [], MP_leftover_idx = []
 Finished with (0, 0) 



Time for decomp of Limb = 22.67707109451294
Number of matching vertices = 39
Only one endpoint after filtering away the endpoints that are not on the skeleton
Total time for Skeletonization and Mesh Correspondence = 494.3785824775696


 ----- Working on Stitching ----------
Total time for stitching floating pieces = 0.009611845016479492


---------Working on soma_idx = 0, soma_group_idx 0, endpt = [1113742.75        543696.30588235  651296.66764706]---------
Starting_edge inside branches_to_conept = [[1113742.75        543696.30588235  651296.66764706]
 [1114120.          543481.          651668.        ]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [56]
printing out current edge:
[[1113742.75        543696.30588235  651296.66764706]
 [1114120.          543481.    

At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [0]
printing out current edge:
[[1123010.  551042.  650049.]
 [1126050.  552268.  647868.]]
edge_endpoints_to_process was empty so exiting loop after 23 iterations
starting_node in concept map (that should match the starting edge) = 0
Total time for branches to concept conversion = 0.2879035472869873

Done generating concept network 


recovered_touching_piece = [0]


---------Working on soma_idx = 0, soma_group_idx 0, endpt = [1108720.  554522.  651422.]---------
Starting_edge inside branches_to_conept = [[1108530.  554747.  651306.]
 [1108720.  554522.  651422.]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [6]
printing out current edge:
[[1108720.  554522.  651422.]
 [1108530.  554747.  651306.]]
edge_endpoints_to_process was empty so exiting loop after 7 iterations
starting_node in concept map (that should match the starting edge) = 4


Working on limb L1 branch 9
Working on limb L1 branch 10
Working on limb L1 branch 51
Working on limb L1 branch 52
Working on limb L1 branch 39
Working on limb L1 branch 53
Assigning the old width calculation because no valid new widths
Working on limb L1 branch 55
Assigning the old width calculation because no valid new widths
Working on limb L1 branch 7
Working on limb L1 branch 11
Working on limb L1 branch 37
Working on limb L1 branch 38
Working on limb L1 branch 49
Assigning the old width calculation because no valid new widths
Working on limb L1 branch 3
Working on limb L1 branch 8
Working on limb L1 branch 14
Working on limb L1 branch 15
Working on limb L1 branch 22
Working on limb L1 branch 35
Working on limb L1 branch 27
Working on limb L1 branch 30
Working on limb L1 branch 0
Working on limb L1 branch 2
Working on limb L1 branch 12
Working on limb L1 branch 13
Working on limb L1 branch 44
Working on limb L1 branch 46
Working on limb L1 branch 34
Working on limb L1 branch 36
Wo

# Debugging Mesh